In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def click_mas():
    # Pinchar en el botón del menu "Más"
    masMenu = driver.find_element(By.XPATH, '//*[@id="content"]/header/div[2]/ul/li[5]/a')

    try:
        masMenu.click()
    except (ElementNotInteractableException, NoSuchElementException):
        # Maneja la excepción y espera antes de intentar nuevamente
        print("Anuncio detectado, reiniciando driver...")
        driver.refresh()
        time.sleep(3) 
        masMenu.click()

In [15]:
# Función para obtener el valor basado en la etiqueta
def obtener_valor_por_etiqueta(label_deseado):
    elemento = driver.find_element(By.XPATH, f"//div[@class='item']//div[@class='label' and text()='{label_deseado}']/following-sibling::div[@class='value']")
    valor = elemento.text
    return valor

In [23]:
# Crea una instancia del controlador del navegador
driver = webdriver.Chrome()

# Navega a la página web que deseas hacer scraping
driver.get("https://mister.mundodeportivo.com/new-onboarding/#market")

# Espera a que se cargue la página
driver.implicitly_wait(15)

# Encuentra el botón de "Consentir" 
button = driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]')
# Haz clic en el botón de "Consentir" 
button.click()

# Encuentra el botón de "Siguinete" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div[2]/button')
# Haz clic en el botón de "Siguiente" 
button.click()
button.click()
button.click()
button.click()

# Encuentra el botón de "sing con gmail" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/button[3]')
button.click()

# Localiza el elemento del input gmail
inputgmail = driver.find_element(By.XPATH, '//*[@id="email"]')

# Borra cualquier contenido existente en la caja de texto (opcional)
inputgmail.clear()

# Ingresa texto en la caja de texto
inputgmail.send_keys("m31_grupo6@outlook.com")

# Localiza el elemento del input gmail
inputpsw = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/form/div[2]/input')

# Borra cualquier contenido existente en la caja de texto (opcional)
inputpsw.clear()

# Ingresa texto en la caja de texto
inputpsw.send_keys("Chocoflakes2")

# Encuentra el botón de "sing con gmail" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/form/div[3]/button')
button.click()

# Espera a que se cargue la página
driver.implicitly_wait(10)

click_mas()

# Pinchar en el botón "Jugaodres" para acceder al listado de jugadores 
jugadoresbtn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/button[2]')

try:
    jugadoresbtn.click()
except (ElementNotInteractableException, NoSuchElementException):
    # Maneja la excepción y espera antes de intentar nuevamente
    print("Anuncio detectado, reiniciando driver...")
    driver.refresh()
    time.sleep(3)
    click_mas()
    time.sleep(3)
    try:
        jugadoresbtn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/button[2]')
        jugadoresbtn.click()
    except: 
        print("Reicicia el script :(")
        sys.exit()
        
pag=2
while True:

    # Encontrar todos los elementos li
    elementos_lis = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")

    # Longitud de la lista de elementos encontrados
    length=len(elementos_lis)

    index=0
    while index < length:
        # Encontrar todos los elementos li
        elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
        elementos_li[index].click()

        time.sleep(2)
        nombre = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[2]")
        apellido = driver.find_element(By.XPATH, " /html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[3]")
        valor=driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div[2]/div[2]/div/div/div[1]/div[2]')
        
        media_puntos_local = obtener_valor_por_etiqueta("Media en casa")
        media_puntos_visitante = obtener_valor_por_etiqueta("Media fuera")
        try:
            edad = obtener_valor_por_etiqueta("Edad")
            altura = obtener_valor_por_etiqueta("Altura")
            peso = obtener_valor_por_etiqueta("Peso")
        except:
            edad = None
            altura = None
            peso = None
            
        if peso == "kg":
            peso = None

        print("____________________________________")
        print(index," -",nombre.text,apellido.text)
        print("Valor: ",valor.text)
        print(f"Media en casa: {media_puntos_local}")
        print(f"Media fuera: {media_puntos_visitante}")
        print(f"Edad: {edad}")
        print(f"Altura: {altura}")
        print(f"Peso: {peso}")
        
        #Retroceder página
        driver.back()
        time.sleep(2)
        elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
        driver.execute_script("arguments[0].scrollIntoView(true);", elementos_li[index])
        time.sleep(2)
        index += 1

    #jugador cambio de pagina
    try:
        ver_mas = driver.find_element(By.XPATH, '/html/body/div[6]/div[3]/div[3]/div[1]/button')
        ver_mas.click()
        time.sleep(4)
    except:
        break

    # Encontrar todos los elementos li
    elementos_lis = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
    # Longitud de la lista de elementos encontrados
    length=len(elementos_lis)

    elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
    elementos_li[index].click()

    time.sleep(1)
 
    #Retroceder página
    driver.back()
     
    print("------------------------------------")
    print("Sigueinte página...  (",pag,")")
    index=1
    pag+=1

driver.quit()    
print("Todos los jugadores scapeados")

____________________________________
0  - Jude Bellingham
Valor:  21.953.000
Media en casa: 10,3
Media fuera: 12,3
Edad: 20
Altura: 1,88m
Peso: None
____________________________________
1  - Takefusa Kubo
Valor:  19.892.000
Media en casa: 10,6
Media fuera: 5,2
Edad: 22
Altura: 1,74m
Peso: 67kg
____________________________________
2  - Antoine Griezmann
Valor:  22.332.000
Media en casa: 8,0
Media fuera: 8,5
Edad: 32
Altura: 1,76m
Peso: 71kg
____________________________________
3  - Aleix García
Valor:  17.762.000
Media en casa: 6,8
Media fuera: 8,3
Edad: 26
Altura: 1,73m
Peso: 61kg
____________________________________
4  -  Isco
Valor:  16.079.000
Media en casa: 8,2
Media fuera: 7,0
Edad: 31
Altura: 1,76m
Peso: 74kg
____________________________________
5  - Ante Budimir
Valor:  11.958.000
Media en casa: 8,0
Media fuera: 6,0
Edad: 32
Altura: 1,90m
Peso: 75kg
____________________________________
6  - Iñaki Williams
Valor:  15.992.000
Media en casa: 6,9
Media fuera: 7,2
Edad: 29
Altura: 1,